In [2]:
# import required libraries
import os
import ansys.fluent.core as pyfluent
from pprint import pprint #debugging library for viewing the state of pyfluent function settings

In [3]:
#### set linux environmnet variables 
# set this to the current installed ANSYS location and version 
# If on Windows comment them out as they are set automatically 
#os.environ['AWP_ROOT222'] = '/opt/packages/ansys/v222' #bridges2 
os.environ['AWP_ROOT242'] = '/usr/ansys_inc/v242' # local 

In [4]:
## BEFORE RUNNING
# what this code needs: a spaceclaim file that has been converted into a mesh file (linux machines cannot read space claim cad files)
# a case file with the modifications you want - you can use a course mesh for this as it will be replaced later
startmesh = 'archimedespremesh.msh.h5' # mesh from spaceclaim, needs to exist
# Mesh specification in SpaceClaim
# DYNAMIC (inner enclosure) - 15 mm dynamic mesh radius and length
# STATIC (outer enclosure) - 1500 mm radius, 5000 mm length either side
#end - 10841.5 mm length, 1821.03 mm radius

startcase = 'archimedesprep.cas.h5' # case file from fluent, needs to exist
#Important note: when preparing a case file, use a coarse mesh generated by pyfluent code below
# this is because the automated code likes to prepend "origin" to the name of everything
# and for consistency when swapping meshes later things need to have the same name

# start mesh and start case need to be in the folder where workdir below points to

In [5]:
# %%
# Set up directories and locations of files

workdir = '/workspaces/ANSYS' 
os.chdir(workdir) # set the working directory where initial files are stored
meshfolder = 'archimedes-screw' # folder where data is saved
meshfile = 'solutionmesh.msh.h5' # mesh to be generated

# create folder where solution files will be stored if it doesn't exist
if not os.path.exists(meshfolder):
    os.mkdir(meshfolder)
finalmeshlocation = os.path.join(meshfolder, meshfile)

In [6]:
# Fluent numerical settings
processors = 8 # set this to the number of cores on the computer
# msinfo in the task bar on windows Windows to check, lscpu on linux

# Meshing Settings
lsize = .01 # size of local sizing mesh in m
ssmol =  lsize # lower bound of surface mesh in m
slarge = .2 # upper bound of surface mesh in m
lrsize = .01 # last ratio size
remesh = False # if at a later time you want to remesh, set this to True

# Solution Settings
gpuUse = False
recalc = False # whether to overwrite solution data with new run or not 
iterations = 1000
velocities = [2.5]

In [7]:
# meshing is only run once and then never called again to save on compute time
# to mesh again, set remesh to True and run all

def meshing(processors, startmesh, meshfolder, meshfile, lsize, ssmol, slarge, lrsize):
    # launch meshing session
    session = pyfluent.launch_fluent(mode="meshing", version="3d", precision="double", processor_count=processors, start_timeout=480)
    workflow = session.workflow
    meshing = session.meshing

    # prepare the mesher
    meshing.GlobalSettings.EnableCleanCAD.set_state(True)
    meshing.GlobalSettings.EnableComplexMeshing.set_state(True)
    meshing.GlobalSettings.UseAllowedValues.set_state(True)
    meshing.GlobalSettings.EnablePrimeMeshing.set_state(False)
    meshing.GlobalSettings.EnablePrime2dMeshing.set_state(False)
    workflow.InitializeWorkflow(WorkflowType=r'Watertight Geometry')

    # set meters as the default unit of measurement
    meshing.GlobalSettings.LengthUnit.set_state(r'm')
    meshing.GlobalSettings.AreaUnit.set_state(r'm^2')
    meshing.GlobalSettings.VolumeUnit.set_state(r'm^3')

    location = [workdir, startmesh]
    # read the spaceclaim mesh file (preconverted into .msh.h5 format)
    workflow.TaskObject['Import Geometry'].Arguments.set_state({r'FileFormat': r'Mesh',r'ImportCadPreferences': {r'CISeparation': r'region',r'MaxFacetLength': 0,r'ShowImportCadPreferences': False,},r'LengthUnit': r'm',r'MeshFileName': r'/'.join(location),})
    workflow.TaskObject['Import Geometry'].Execute()

    workflow.TaskObject['Add Local Sizing'].Arguments.set_state({r'AddChild': r'yes',r'BOICellsPerGap': 1,r'BOIControlName': r'bodysize_1',r'BOICurvatureNormalAngle': 18,r'BOIExecution': r'Body Size',r'BOIFaceLabelList': [r'dynamic'],r'BOIGrowthRate': 1.2,r'BOISize': lsize,r'BOIZoneorLabel': r'label',})
    workflow.TaskObject['Add Local Sizing'].AddChildAndUpdate(DeferUpdate=False)

    workflow.TaskObject['Generate the Surface Mesh'].Arguments.set_state({r'CFDSurfaceMeshControls': {r'MaxSize': slarge,r'MinSize': ssmol,},})
    workflow.TaskObject['Generate the Surface Mesh'].Execute()

    workflow.TaskObject['Describe Geometry'].UpdateChildTasks(SetupTypeChanged=False)
    workflow.TaskObject['Describe Geometry'].Arguments.set_state({r'NonConformal': r'No',r'SetupType': r'The geometry consists of only fluid regions with no voids',})
    workflow.TaskObject['Describe Geometry'].UpdateChildTasks(SetupTypeChanged=True)
    workflow.TaskObject['Describe Geometry'].Arguments.set_state({r'NonConformal': r'No',r'SetupType': r'The geometry consists of only fluid regions with no voids',r'WallToInternal': r'Yes',})
    workflow.TaskObject['Describe Geometry'].Execute()

    workflow.TaskObject['Update Boundaries'].Arguments.set_state({r'BoundaryLabelList': [r'inlet'],r'BoundaryLabelTypeList': [r'pressure-outlet'],r'OldBoundaryLabelList': [r'inlet'],r'OldBoundaryLabelTypeList': [r'velocity-inlet'],})
    workflow.TaskObject['Update Boundaries'].Execute()

    workflow.TaskObject['Update Regions'].Arguments.set_state({r'OldRegionNameList': [r'fluid'],r'OldRegionTypeList': [r'fluid'],r'RegionNameList': [r'fluid'],r'RegionTypeList': [r'dead'],})
    workflow.TaskObject['Update Regions'].Execute()

    workflow.TaskObject['Add Boundary Layers'].Arguments.set_state({r'BLControlName': r'last-ratio_1',r'BLRegionList': [r'dynamic'],r'BLZoneList': [r'wall'],r'CompleteRegionScope': [r'dynamic'],r'FaceScope': {r'RegionsType': r'named-regions',},r'FirstHeight': lrsize,r'LocalPrismPreferences': {r'Continuous': r'Continuous',},r'OffsetMethodType': r'last-ratio',r'RegionScope': [r'dynamic'],})
    workflow.TaskObject['Add Boundary Layers'].AddChildAndUpdate(DeferUpdate=False)

    workflow.TaskObject['Generate the Volume Mesh'].Arguments.set_state({r'VolumeFill': r'poly-hexcore',r'VolumeMeshPreferences': {r'Avoid1_8Transition': r'yes',r'ShowVolumeMeshPreferences': True,}})
    workflow.TaskObject['Generate the Volume Mesh'].Execute()

    # write mesh file to selected folder for reuse and easy import to case later
    meshing.File.WriteMesh(FileName=finalmeshlocation)

    #close fluent once meshing done
    session.exit()

In [ ]:
def solving(processors, gpuUse, recalc, iterations, startcase, velocities, finalmeshlocation, meshfolder):
    # relaunch fluent as solution mode
    solver = pyfluent.launch_fluent(mode="solver", version="3d", precision="double", processor_count=processors, gpu = gpuUse, start_timeout=360)

    # read case file 
    solver.file.read_case(file_type="case", file_name=startcase)

    # replace the mesh with the fine mesh just generated
    solver.file.replace_mesh(file_type="mesh", file_name=finalmeshlocation)

    # surfaces we want data from:
    surfaces = ["origin-wall", "origin-inlet", "origin-outlet"]

    for  velocity in velocities:
        datapath = os.path.join(meshfolder, str(velocity))
        if not os.path.exists(datapath) or recalc:
            os.mkdir(datapath)
            dataname = os.path.join(datapath, str(velocity))

            # set up the capture of the output
            %%capture moment_output
            solver.setup.boundary_conditions.pressure_inlet['origin-inlet'] = {"phase" : {"mixture" : {"multiphase" : {"vmag" : {"value" : velocity}}}}}
            #initialize 
            #solver.initialization.open_channel_auto_init = {"boundary_zone": "origin-inlet", "flat_init": True}
            #solver.initialization.open_channel_auto_init.flat_init=True
            #solver.tui.initialize()
            solution = solver.settings.solution
            solution.initialization.hybrid_initialize()
            pprint(solution.initialization.hybrid_initialize())
            solution.run_calculation.iterate(iter_count=iterations)

            # save output (moment data)
            with open(os.path.join(datapath, "moments" + str(velocity) + ".txt"), "w") as f:
                f.write(moment_output.stdout)

            # use pprint to know what data can be read from the solver object
            #pprint(solver.settings.file.export.ascii.cell_func_domain.allowed_values())

            # save all the results in between iterations 
            solver.file.write_data(file_name=dataname)
            datatoread = ['velocity-magnitude', 'x-velocity', 'y-velocity', 'z-velocity', 'axial-velocity', 'radial-velocity', 'tangential-velocity', 
                        'dx-velocity-dx', 'dy-velocity-dx', 'dz-velocity-dx', 'dx-velocity-dy', 'dy-velocity-dy', 
                        'dz-velocity-dy', 'dx-velocity-dz', 'dy-velocity-dz', 'dz-velocity-dz']
            for surface in surfaces:
                solver.settings.file.export.ascii(file_name = "".join([dataname, surface, ".txt"]), surface_name_list = [surface], cell_func_domain = datatoread)
            solver.settings.file.write_case(file_name=os.path.join(datapath, "archimedes-solutioncase.cas.h5"))
    solver.exit()

In [9]:
if (not os.path.exists(finalmeshlocation) or remesh): #only remesh if desired
    meshing(processors, meshfolder, meshfile, lsize, ssmol, slarge, lrsize)
else:
    print("Mesh already exists")
solving(processors, gpuUse, recalc, iterations, startcase, velocities, finalmeshlocation, meshfolder)

# DONE

/home/vscode/.local/lib/python3.10/site-packages/ansys/fluent/core/utils/deprecate.py:72: PyFluentDeprecationWarning: 'version' is deprecated. Use 'dimension' instead.
  return func(*args, **kwargs)
pyfluent.general WARNING: Using 'dimension = Dimension.THREE' for 'launch_fluent()' instead of 'version = 3d'.


Mesh already exists
/usr/ansys_inc/v242/fluent/fluent24.2.0/bin/fluent -r24.2.0 3ddp -t8 -gu -sifile=/tmp/serverinfo-unhjdnz8.txt -nm
/usr/ansys_inc/v242/fluent/fluent24.2.0/cortex/lnamd64/cortex.24.2.0 -f fluent -gu -sifile=/tmp/serverinfo-unhjdnz8.txt -nm (fluent "3ddp -pdefault  -host -r24.2.0 -t8 -mpi=intel -path/usr/ansys_inc/v242/fluent -ssh")


sh: 1: glxinfo: not found


KeyboardInterrupt: 